## Start Desktop

In [2]:
from ansys.aedt.core import Desktop, Maxwell3d, Hfss
import ansys.aedt.core.downloads as downloads
import os,time,json


# Launch AEDT
AedtVersion = "2024.1"  # Replace with your installed AEDT version
ProjectName = "HBCPMProject"
DesignName = "HBCPM"
ProjectPath=os.path.join(os.getcwd(), f"{ProjectName}.aedt")

desktop = Desktop(version=AedtVersion,new_desktop=True, non_graphical=False, close_on_exit=True)
print(desktop.odesktop)

HBCPM = Maxwell3d(
                design=DesignName,
                solution_type="",
                version=AedtVersion,
                new_desktop=True, 
                non_graphical=False, 
                close_on_exit=True)

HBCPM.solution_type = HBCPM.SOLUTIONS.Maxwell3d.Magnetostatic

oProject =desktop.odesktop.GetActiveProject()
oProject.Rename("C:/he/HBCPM/HBCPMProject.aedt", True)
oDesign = oProject.GetActiveDesign()

oEditor = oDesign.SetActiveEditor("3D Modeler")

print(oDesign)
print(oProject)
print(oEditor)

PyAEDT INFO: Python version 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:07:43) [MSC v.1942 64 bit (AMD64)]
PyAEDT INFO: PyAEDT version 0.12.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\shins\AppData\Local\Temp\pyaedt_shins_6c804f6f-4a47-4bfa-b720-5fa48f7178f1.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 60400
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v241\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.1 version started with process ID 29620.
Instance of an Aedt object:13
PyAEDT INFO: Python version 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:07:43) [MSC v.1942 64 bit (AMD64)]
PyAEDT INFO: PyAEDT version 0.12.0.
PyAEDT INFO: Returning found Desktop session with PID 29620!
PyAE

### Define parameters and materials

In [3]:
# Define variables and expressions in a dictionary
variables = {
    "RotorInnerRadius": "16.6mm",
    "RotorCenterThickness": "8mm",
    "RotorOuterRadius": "25mm",
    "RadialPMThickness": "2mm",
    "RadialPMAngle": "38deg",
    "RotorPMAxialThickness": "2mm",
    "RotorPMAxialOuterRadius": "25mm-RadialPMThickness",
    "RotorIronOuterRadius": "25mm",
    "RotorIronThickness": "1.5mm",
    "StatorYokeWidth": "8mm",
    "StatorInnerRadius": "27mm",
    "StatorAxialThickness": "8mm",
    "StatorOuterRadius": "56mm",
    "StatorPoleLength": "8.5mm",
    "StatorPoleWidth": "8mm",
    "StatorPMOuterRadius": "29.5mm",
    "StatorPMThickness": "2mm",
    "StatorIronThickness": "1.5mm",
    "StatorIronOuterRadius": "29mm",
    "RotorPMInnerRadius": "RotorPMAxialOuterRadius-3mm",
    "RotorIronInnerRadius": "RotorPMInnerRadius+0.5mm",
    "WindingThickness": "3.2mm",
    "WindingRadialLength": "13.3mm",
    "rpm": "3000",
    "StatorPoleTeethAdditionLength": "2mm",
    "StatorPoleArcTang": "1mm",
    "turnm": "90",
    "turns": "100",
    "Im": "2A",
    "ImA": "Im*cos(rpm/60*2*pi*time*4+pi/2)",
    "ImB": "Im*cos(rpm/60*2*pi*time*4+pi/2-2*pi/3)",
    "ImC": "Im*cos(rpm/60*2*pi*time*4+pi/2+2*pi/3)",
}

# Save the dictionary as a JSON file
with open("Para.json", "w") as json_file:
    json.dump(variables, json_file, indent=4)

print("Parameters saved to Para.json successfully!")

# This call returns the VariableManager class
# Iterate through the dictionary and set each variable
for var_name, expression in variables.items():
    HBCPM.variable_manager.set_variable(var_name, expression=expression)

#  define new materials

oDefinitionManager = oProject.GetDefinitionManager()

oDefinitionManager.AddMaterial(
	[
		"NAME:TDK_NEOREC40TH_60cel_Radial",
		"CoordinateSystemType:=", "Cylindrical",
		"BulkOrSurfaceType:="	, 1,
		[
			"NAME:PhysicsTypes",
			"set:="			, ["Electromagnetic"]
		],
		"permeability:="	, "1.04035644080587",
		"conductivity:="	, "769230.769",
		[
			"NAME:magnetic_coercivity",
			"property_type:="	, "VectorProperty",
			"Magnitude:="		, "-956678.252234359A_per_meter",
			"DirComp1:="		, "1",
			"DirComp2:="		, "0",
			"DirComp3:="		, "0",
		]
	])

oDefinitionManager.AddMaterial(
	[
		"NAME:TDK_NEOREC40TH_60cel_Down",
		"CoordinateSystemType:=", "Cartesian",
		"BulkOrSurfaceType:="	, 1,
		[
			"NAME:PhysicsTypes",
			"set:="			, ["Electromagnetic"]
		],
		"permeability:="	, "1.04035644080587",
		"conductivity:="	, "769230.769",
		[
			"NAME:magnetic_coercivity",
			"property_type:="	, "VectorProperty",
			"Magnitude:="		, "-956678.252234359A_per_meter",
			"DirComp1:="		, "0",
			"DirComp2:="		, "0",
			"DirComp3:="		, "-1"
		]
	])

oDefinitionManager.AddMaterial(
	[
		"NAME:TDK_NEOREC40TH_60cel_Up",
		"CoordinateSystemType:=", "Cartesian",
		"BulkOrSurfaceType:="	, 1,
		[
			"NAME:PhysicsTypes",
			"set:="			, ["Electromagnetic"]
		],
		"permeability:="	, "1.04035644080587",
		"conductivity:="	, "769230.769",
		[
			"NAME:magnetic_coercivity",
			"property_type:="	, "VectorProperty",
			"Magnitude:="		, "-956678.252234359A_per_meter",
			"DirComp1:="		, "0",
			"DirComp2:="		, "0",
			"DirComp3:="		, "1"
		]
	])


'TDK_NEOREC40TH_60cel_Up'

### Model the motor rotor parts
- define the object in Create, Duplicate function
- the object is actually the name or namelist of the each instance

In [4]:

# Rotor main body
###################################################################################

Rotor = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "RotorInnerRadius",
		"YStart:="		, "-RotorCenterThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "RotorOuterRadius-RotorInnerRadius",
		"Height:="		, "RotorCenterThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "Rotor",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"JFE_Steel_35JNE300\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, Rotor,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

# Rotor radial PM
RotorRadialPM = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "RotorOuterRadius-RadialPMThickness",
		"YStart:="		, "-RotorCenterThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "RadialPMThickness",
		"Height:="		, "RotorCenterThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "RotorRadialPM",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"TDK_NEOREC40TH_60cel_Radial\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, RotorRadialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "RadialPMAngle/2",
		"NumOfSegments:="	, "0"
	])

RotorRadialPMhalf = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, RotorRadialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "0mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "0mm",
		"DuplicateMirrorNormalZ:=", "1mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

RotorRadialPMhalfList=[RotorRadialPM]
RotorRadialPMhalfList.extend(RotorRadialPMhalf)

oEditor.Unite(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, RotorRadialPMhalfList))
	], 
	[
		"NAME:UniteParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

RotorRadialPMDupt = oEditor.DuplicateAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, RotorRadialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateAroundAxisParameters",
		"CreateNewObjects:="	, True,
		"WhichAxis:="		, "Y",
		"AngleStr:="		, "90deg",
		"NumClones:="		, "4"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

#  define all object of PM
RotorRadialPMList=[RotorRadialPM]
RotorRadialPMList.extend(RotorRadialPMDupt)
# print(RotorRadialPMList)

oEditor.Subtract(
	[
		"NAME:Selections",
		"Blank Parts:="		, "Rotor",
		"Tool Parts:="		, ",".join(map(str, RotorRadialPMList))
	], 
	[
		"NAME:SubtractParameters",
		"KeepOriginals:="	, True,
		"TurnOnNBodyBoolean:="	, True
	])

print("Create "+ Rotor + " successful")

# Rotor axial PM
RotorAxialPM = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "RotorPMInnerRadius",
		"YStart:="		, "RotorCenterThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "RotorPMAxialOuterRadius-RotorPMInnerRadius",
		"Height:="		, "RotorPMAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "RotorAxialPM",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"TDK_NEOREC40TH_60cel_Up\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, RotorAxialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

RotorAxialPMDupt = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, RotorAxialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "0mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "-1mm",
		"DuplicateMirrorNormalZ:=", "0mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

RotorAxialPMList=[RotorAxialPM]

RotorAxialPMList.extend(RotorAxialPMDupt)
# print(RotorAxialPMList)

oEditor.ChangeProperty(
	[
		"NAME:AllTabs",
		[
			"NAME:Geometry3DAttributeTab",
			[
				"NAME:PropServers", 
				RotorAxialPMList[1], 
			],
			[
				"NAME:ChangedProps",
				[
					"NAME:Material",
					"Value:="		, "\"TDK_NEOREC40TH_60cel_Down\""
				]
			]
		]
	])

print("Create "+ str(RotorAxialPMList) + " successful")

# Rotor axial Iron
RotorAxialIron = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "RotorIronOuterRadius",
		"YStart:="		, "RotorCenterThickness/2+RotorPMAxialThickness",
		"ZStart:="		, "0mm",
		"Width:="		, "-(RotorIronOuterRadius-RotorIronInnerRadius)",
		"Height:="		, "RotorIronThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "RotorAxialIron",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"iron\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, RotorAxialIron,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

RotorAxialIronDupt = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, RotorAxialIron,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "25mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "-1mm",
		"DuplicateMirrorNormalZ:=", "0mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

RotorAxialIronList=[RotorAxialIron]
RotorAxialIronList.extend(RotorAxialIronDupt)

print("Create " + str(RotorAxialIronList) + " successful")

###################################################################################

Create Rotor successful
Create ['RotorAxialPM', 'RotorAxialPM_1'] successful
Create ['RotorAxialIron', 'RotorAxialIron_1'] successful


### Model the motor stator parts

In [5]:

# Stator axial PM
StatorAxialPM = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "StatorInnerRadius",
		"YStart:="		, "StatorAxialThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorPMOuterradius-StatorInnerRadius",
		"Height:="		, "StatorPMThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorAxialPM",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"TDK_NEOREC40TH_60cel_Down\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, StatorAxialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

StatorAxialPMDupt = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		,  StatorAxialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "0mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "-1mm",
		"DuplicateMirrorNormalZ:=", "0mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

StatorAxialPMList=[StatorAxialPM]

StatorAxialPMList.extend(StatorAxialPMDupt)
# print(RotorAxialPMList)

oEditor.ChangeProperty(
	[
		"NAME:AllTabs",
		[
			"NAME:Geometry3DAttributeTab",
			[
				"NAME:PropServers", 
				StatorAxialPMList[1], 
			],
			[
				"NAME:ChangedProps",
				[
					"NAME:Material",
					"Value:="		, "\"TDK_NEOREC40TH_60cel_Down\""
				]
			]
		]
	])

print("Create "+ str(StatorAxialPMList) + " successful")


# Stator axial Iron
StatorAxialIron = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "StatorInnerRadius",
		"YStart:="		, "StatorAxialThickness/2+StatorPMThickness",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorIronOuterRadius-StatorInnerRadius",
		"Height:="		, "StatorIronThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorAxialIron",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"iron\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, StatorAxialIron,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

StatorAxialIronDupt = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, StatorAxialIron,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "25mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "-1mm",
		"DuplicateMirrorNormalZ:=", "0mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

StatorAxialIronList=[StatorAxialIron]
StatorAxialIronList.extend(StatorAxialIronDupt)

print("Create " + str(StatorAxialIron) + " successful")


# Create stator Pole and Yoke


# Stator Yoke
Stator = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "StatorOuterRadius-StatorYokeWidth",
		"YStart:="		, "-StatorAxialThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorYokeWidth",
		"Height:="		, "StatorAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorYoke",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"JFE_Steel_35JNE300\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, Stator,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

print("Create " + str(Stator) + " successful")

# Stator Pole
StatorPole = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "0mm",
		"YStart:="		, "-StatorAxialThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorOuterRadius",
		"Height:="		, "StatorAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorPole",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"JFE_Steel_35JNE300\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAlongVector(
	[
		"NAME:Selections",
		"Selections:="		, StatorPole,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:VectorSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepVectorX:="	, "0mm",
		"SweepVectorY:="	, "0mm",
		"SweepVectorZ:="	, "StatorPoleWidth/2"
	])



StatorPolehalfDupt = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, StatorPole,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "-4mm",
		"DuplicateMirrorBaseZ:=", "0mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "0mm",
		"DuplicateMirrorNormalZ:=", "-1mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])
# return StatorPolehalfDupt is a list

StatorPoleList=[StatorPole]

StatorPoleList.extend(StatorPolehalfDupt)

oEditor.Unite(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, StatorPoleList))
	], 
	[
		"NAME:UniteParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

StatorPoleIntersector = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "StatorInnerRadius",
		"YStart:="		, "-StatorAxialThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorOuterRadius-StatorInnerRadius-StatorYokeWidth",
		"Height:="		, "StatorAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorPoleIntersector",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])
# return StatorPoleIntersector is a single str

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, StatorPoleIntersector,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

StatorPoleIntersectorList=[StatorPole,StatorPoleIntersector]
# print(StatorPoleIntersectorList)

oEditor.Intersect(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, StatorPoleIntersectorList))
	], 
	[
		"NAME:IntersectParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

print("Create " + str(StatorPole) + " successful")

# Stator tooth
StatorTooth = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "0mm",
		"YStart:="		, "-StatorAxialThickness/2",
		"ZStart:="		, "StatorPoleWidth/2",
		"Width:="		, "StatorOuterRadius",
		"Height:="		, "StatorAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorTooth",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"JFE_Steel_35JNE300\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAlongVector(
	[
		"NAME:Selections",
		"Selections:="		, "StatorTooth",
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:VectorSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepVectorX:="	, "0mm",
		"SweepVectorY:="	, "0mm",
		"SweepVectorZ:="	, "StatorPoleTeethAdditionLength"
	])

StatorToothIntersector = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "StatorInnerRadius",
		"YStart:="		, "-StatorAxialThickness/2",
		"ZStart:="		, "0",
		"Width:="		, "StatorPMOuterRadius-StatorInnerRadius",
		"Height:="		, "StatorAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorToothIntersector",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, StatorToothIntersector,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

StatorToothList=[StatorTooth,StatorToothIntersector]

oEditor.Intersect(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, StatorToothList))
	], 
	[
		"NAME:IntersectParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])
        
print("Create " + str(StatorTooth) + " successful")

StatorToothDul=oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, StatorTooth,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "0mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "0mm",
		"DuplicateMirrorNormalZ:=", "-1mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

StatorToothUnionList = [StatorPole,StatorTooth]
StatorToothUnionList.extend(StatorToothDul)

print(StatorToothUnionList)

oEditor.Unite(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, StatorToothUnionList))
	], 
	[
		"NAME:UniteParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

StatorToothUnion = oEditor.DuplicateAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, StatorToothUnionList[0],
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateAroundAxisParameters",
		"CreateNewObjects:="	, True,
		"WhichAxis:="		, "Y",
		"AngleStr:="		, "30deg",
		"NumClones:="		, "12"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

StatorList=[Stator,StatorToothUnionList[0]]
StatorList.extend(StatorToothUnion)

oEditor.Unite(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, StatorList))
	], 
	[
		"NAME:UniteParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

print("Create " + str(StatorList) + " successful")


Create ['StatorAxialPM', 'StatorAxialPM_1'] successful


In [10]:
desktop.save_project()
desktop.release_desktop()

PyAEDT INFO: Desktop has been released and closed.


True

In [11]:
from ansys.aedt.core import Desktop, Maxwell3d, Hfss
import ansys.aedt.core.downloads as downloads
import os,time,json

# Launch AEDT
AedtVersion = "2024.1"  # Replace with your installed AEDT version
ProjectName = "HBCPMipynb"
DesignName = "HBCPM"
ProjectPath=os.path.join(os.getcwd(), f"{ProjectName}.aedt")

desktop = Desktop(version=AedtVersion,new_desktop=True, non_graphical=False, close_on_exit=True)
print(desktop.odesktop)

HBCPM = Maxwell3d(
                design=DesignName,
                solution_type="",
                version=AedtVersion,
                new_desktop=True, 
                non_graphical=False, 
                close_on_exit=True)

HBCPM.solution_type = HBCPM.SOLUTIONS.Maxwell3d.Magnetostatic

oProject =desktop.odesktop.GetActiveProject()
oProject.Rename("C:/he/HBCPM/HBCPMProject.aedt", True)
oDesign = oProject.GetActiveDesign()

oEditor = oDesign.SetActiveEditor("3D Modeler")

print(oDesign)
print(oProject)
print(oEditor)

# Define variables and expressions in a dictionary
variables = {
    "RotorInnerRadius": "16.6mm",
    "RotorCenterThickness": "8mm",
    "RotorOuterRadius": "25mm",
    "RadialPMThickness": "2mm",
    "RadialPMAngle": "38deg",
    "RotorPMAxialThickness": "2mm",
    "RotorPMAxialOuterRadius": "25mm-RadialPMThickness",
    "RotorIronOuterRadius": "25mm",
    "RotorIronThickness": "1.5mm",
    "StatorYokeWidth": "8mm",
    "StatorInnerRadius": "27mm",
    "StatorAxialThickness": "8mm",
    "StatorOuterRadius": "56mm",
    "StatorPoleLength": "8.5mm",
    "StatorPoleWidth": "8mm",
    "StatorPMOuterRadius": "29.5mm",
    "StatorPMThickness": "2mm",
    "StatorIronThickness": "1.5mm",
    "StatorIronOuterRadius": "29mm",
    "RotorPMInnerRadius": "RotorPMAxialOuterRadius-3mm",
    "RotorIronInnerRadius": "RotorPMInnerRadius+0.5mm",
    "WindingThickness": "3.2mm",
    "WindingRadialLength": "13.3mm",
    "rpm": "3000",
    "StatorPoleTeethAdditionLength": "2mm",
    "StatorPoleArcTang": "1mm",
    "turnm": "90",
    "turns": "100",
    "Im": "2A",
    "ImA": "Im*cos(rpm/60*2*pi*time*4+pi/2)",
    "ImB": "Im*cos(rpm/60*2*pi*time*4+pi/2-2*pi/3)",
    "ImC": "Im*cos(rpm/60*2*pi*time*4+pi/2+2*pi/3)",
    "StatorPoleTeethStartX": "sqrt(StatorPMOuterRadius^2-(StatorPoleWidth/2)^2)meter",
    "StatorPoleTeethAngle": "45deg",
}

# Save the dictionary as a JSON file
with open("Para.json", "w") as json_file:
    json.dump(variables, json_file, indent=4)

print("Parameters saved to Para.json successfully!")

# This call returns the VariableManager class
# Iterate through the dictionary and set each variable
for var_name, expression in variables.items():
    HBCPM.variable_manager.set_variable(var_name, expression=expression)


#  define new materials

oDefinitionManager = oProject.GetDefinitionManager()

oDefinitionManager.AddMaterial(
	[
		"NAME:TDK_NEOREC40TH_60cel_Radial",
		"CoordinateSystemType:=", "Cylindrical",
		"BulkOrSurfaceType:="	, 1,
		[
			"NAME:PhysicsTypes",
			"set:="			, ["Electromagnetic"]
		],
		"permeability:="	, "1.04035644080587",
		"conductivity:="	, "769230.769",
		[
			"NAME:magnetic_coercivity",
			"property_type:="	, "VectorProperty",
			"Magnitude:="		, "-956678.252234359A_per_meter",
			"DirComp1:="		, "1",
			"DirComp2:="		, "0",
			"DirComp3:="		, "0",
		]
	])

oDefinitionManager.AddMaterial(
	[
		"NAME:TDK_NEOREC40TH_60cel_Down",
		"CoordinateSystemType:=", "Cartesian",
		"BulkOrSurfaceType:="	, 1,
		[
			"NAME:PhysicsTypes",
			"set:="			, ["Electromagnetic"]
		],
		"permeability:="	, "1.04035644080587",
		"conductivity:="	, "769230.769",
		[
			"NAME:magnetic_coercivity",
			"property_type:="	, "VectorProperty",
			"Magnitude:="		, "-956678.252234359A_per_meter",
			"DirComp1:="		, "0",
			"DirComp2:="		, "0",
			"DirComp3:="		, "-1"
		]
	])

oDefinitionManager.AddMaterial(
	[
		"NAME:TDK_NEOREC40TH_60cel_Up",
		"CoordinateSystemType:=", "Cartesian",
		"BulkOrSurfaceType:="	, 1,
		[
			"NAME:PhysicsTypes",
			"set:="			, ["Electromagnetic"]
		],
		"permeability:="	, "1.04035644080587",
		"conductivity:="	, "769230.769",
		[
			"NAME:magnetic_coercivity",
			"property_type:="	, "VectorProperty",
			"Magnitude:="		, "-956678.252234359A_per_meter",
			"DirComp1:="		, "0",
			"DirComp2:="		, "0",
			"DirComp3:="		, "1"
		]
	])


# Create 3D model
# define the object in Create, Duplicate function
# the object is actually the name or namelist of the each instance

# Rotor main body
###################################################################################

Rotor = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "RotorInnerRadius",
		"YStart:="		, "-RotorCenterThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "RotorOuterRadius-RotorInnerRadius",
		"Height:="		, "RotorCenterThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "Rotor",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"JFE_Steel_35JNE300\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, Rotor,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

# Rotor radial PM
RotorRadialPM = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "RotorOuterRadius-RadialPMThickness",
		"YStart:="		, "-RotorCenterThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "RadialPMThickness",
		"Height:="		, "RotorCenterThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "RotorRadialPM",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"TDK_NEOREC40TH_60cel_Radial\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, RotorRadialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "RadialPMAngle/2",
		"NumOfSegments:="	, "0"
	])

RotorRadialPMhalf = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, RotorRadialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "0mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "0mm",
		"DuplicateMirrorNormalZ:=", "1mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

RotorRadialPMhalfList=[RotorRadialPM]
RotorRadialPMhalfList.extend(RotorRadialPMhalf)

oEditor.Unite(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, RotorRadialPMhalfList))
	], 
	[
		"NAME:UniteParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

RotorRadialPMDupt = oEditor.DuplicateAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, RotorRadialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateAroundAxisParameters",
		"CreateNewObjects:="	, True,
		"WhichAxis:="		, "Y",
		"AngleStr:="		, "90deg",
		"NumClones:="		, "4"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

#  define all object of PM
RotorRadialPMList=[RotorRadialPM]
RotorRadialPMList.extend(RotorRadialPMDupt)
# print(RotorRadialPMList)

oEditor.Subtract(
	[
		"NAME:Selections",
		"Blank Parts:="		, "Rotor",
		"Tool Parts:="		, ",".join(map(str, RotorRadialPMList))
	], 
	[
		"NAME:SubtractParameters",
		"KeepOriginals:="	, True,
		"TurnOnNBodyBoolean:="	, True
	])

print("Create "+ Rotor + " successful")

# Rotor axial PM
RotorAxialPM = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "RotorPMInnerRadius",
		"YStart:="		, "RotorCenterThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "RotorPMAxialOuterRadius-RotorPMInnerRadius",
		"Height:="		, "RotorPMAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "RotorAxialPM",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"TDK_NEOREC40TH_60cel_Up\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, RotorAxialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

RotorAxialPMDupt = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, RotorAxialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "0mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "-1mm",
		"DuplicateMirrorNormalZ:=", "0mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

RotorAxialPMList=[RotorAxialPM]

RotorAxialPMList.extend(RotorAxialPMDupt)
# print(RotorAxialPMList)

oEditor.ChangeProperty(
	[
		"NAME:AllTabs",
		[
			"NAME:Geometry3DAttributeTab",
			[
				"NAME:PropServers", 
				RotorAxialPMList[1], 
			],
			[
				"NAME:ChangedProps",
				[
					"NAME:Material",
					"Value:="		, "\"TDK_NEOREC40TH_60cel_Down\""
				]
			]
		]
	])

print("Create "+ str(RotorAxialPMList) + " successful")

# Rotor axial Iron
RotorAxialIron = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "RotorIronOuterRadius",
		"YStart:="		, "RotorCenterThickness/2+RotorPMAxialThickness",
		"ZStart:="		, "0mm",
		"Width:="		, "-(RotorIronOuterRadius-RotorIronInnerRadius)",
		"Height:="		, "RotorIronThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "RotorAxialIron",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"iron\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, RotorAxialIron,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

RotorAxialIronDupt = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, RotorAxialIron,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "25mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "-1mm",
		"DuplicateMirrorNormalZ:=", "0mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

RotorAxialIronList=[RotorAxialIron]
RotorAxialIronList.extend(RotorAxialIronDupt)

print("Create " + str(RotorAxialIronList) + " successful")

###################################################################################

# Stator axial PM
StatorAxialPM = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "StatorInnerRadius",
		"YStart:="		, "StatorAxialThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorPMOuterradius-StatorInnerRadius",
		"Height:="		, "StatorPMThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorAxialPM",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"TDK_NEOREC40TH_60cel_Down\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, StatorAxialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

StatorAxialPMDupt = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		,  StatorAxialPM,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "0mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "-1mm",
		"DuplicateMirrorNormalZ:=", "0mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

StatorAxialPMList=[StatorAxialPM]

StatorAxialPMList.extend(StatorAxialPMDupt)
# print(RotorAxialPMList)

oEditor.ChangeProperty(
	[
		"NAME:AllTabs",
		[
			"NAME:Geometry3DAttributeTab",
			[
				"NAME:PropServers", 
				StatorAxialPMList[1], 
			],
			[
				"NAME:ChangedProps",
				[
					"NAME:Material",
					"Value:="		, "\"TDK_NEOREC40TH_60cel_Up\""
				]
			]
		]
	])

print("Create "+ str(StatorAxialPMList) + " successful")


# Stator axial Iron
StatorAxialIron = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "StatorInnerRadius",
		"YStart:="		, "StatorAxialThickness/2+StatorPMThickness",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorIronOuterRadius-StatorInnerRadius",
		"Height:="		, "StatorIronThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorAxialIron",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"iron\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, StatorAxialIron,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

StatorAxialIronDupt = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, StatorAxialIron,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "25mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "-1mm",
		"DuplicateMirrorNormalZ:=", "0mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

StatorAxialIronList=[StatorAxialIron]
StatorAxialIronList.extend(StatorAxialIronDupt)

print("Create " + str(StatorAxialIron) + " successful")


# Create stator Pole and Yoke

# Stator Yoke
Stator = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "StatorOuterRadius-StatorYokeWidth",
		"YStart:="		, "-StatorAxialThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorYokeWidth",
		"Height:="		, "StatorAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "Stator",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"JFE_Steel_35JNE300\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, Stator,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

print("Create " + str(Stator) + " successful")

# Stator Pole
StatorPole = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "0mm",
		"YStart:="		, "-StatorAxialThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorOuterRadius",
		"Height:="		, "StatorAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorPole",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"JFE_Steel_35JNE300\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAlongVector(
	[
		"NAME:Selections",
		"Selections:="		, StatorPole,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:VectorSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepVectorX:="	, "0mm",
		"SweepVectorY:="	, "0mm",
		"SweepVectorZ:="	, "StatorPoleWidth/2"
	])



StatorPolehalfDupt = oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, StatorPole,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "-4mm",
		"DuplicateMirrorBaseZ:=", "0mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "0mm",
		"DuplicateMirrorNormalZ:=", "-1mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])
# return StatorPolehalfDupt is a list

StatorPoleList=[StatorPole]

StatorPoleList.extend(StatorPolehalfDupt)

oEditor.Unite(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, StatorPoleList))
	], 
	[
		"NAME:UniteParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

StatorPoleIntersector = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "StatorInnerRadius",
		"YStart:="		, "-StatorAxialThickness/2",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorOuterRadius-StatorInnerRadius-StatorYokeWidth",
		"Height:="		, "StatorAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorPoleIntersector",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])
# return StatorPoleIntersector is a single str

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, StatorPoleIntersector,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

StatorPoleIntersectorList=[StatorPole,StatorPoleIntersector]
# print(StatorPoleIntersectorList)

oEditor.Intersect(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, StatorPoleIntersectorList))
	], 
	[
		"NAME:IntersectParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

print("Create " + str(StatorPole) + " successful")

# Stator tooth
StatorTooth = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "0mm",
		"YStart:="		, "-StatorAxialThickness/2",
		"ZStart:="		, "StatorPoleWidth/2",
		"Width:="		, "StatorOuterRadius",
		"Height:="		, "StatorAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorTooth",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"JFE_Steel_35JNE300\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAlongVector(
	[
		"NAME:Selections",
		"Selections:="		, "StatorTooth",
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:VectorSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepVectorX:="	, "0mm",
		"SweepVectorY:="	, "0mm",
		"SweepVectorZ:="	, "StatorPoleTeethAdditionLength"
	])

StatorToothIntersector1 = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "StatorInnerRadius",
		"YStart:="		, "-StatorAxialThickness/2",
		"ZStart:="		, "0",
		"Width:="		, "StatorPMOuterRadius-StatorInnerRadius",
		"Height:="		, "StatorAxialThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorToothIntersector1",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, StatorToothIntersector1,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:AxisSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepAxis:="		, "Y",
		"SweepAngle:="		, "360deg",
		"NumOfSegments:="	, "0"
	])

StatorToothIntersector2 = oEditor.CreatePolyline(
	[
		"NAME:PolylineParameters",
		"IsPolylineCovered:="	, True,
		"IsPolylineClosed:="	, False,
		[
			"NAME:PolylinePoints",
			[
				"NAME:Point1",
				"X:="			, "StatorPoleTeethStartX",
				"Y:="			, "-StatorAxialThickness/2",
				"Z:="			, "StatorPoleWidth/2"
			],
			[
				"NAME:Point2",
				"X:="			, "StatorPoleTeethStartX",
				"Y:="			, "-StatorAxialThickness/2",
				"Z:="			, "20mm"

			]
		],
		[
			"NAME:PolylineSegments",
			[
				"NAME:PLSegment",
				"SegmentType:="		, "Line",
				"StartIndex:="		, 0,
				"NoOfPoints:="		, 2
			]
		],
		[
			"NAME:PolylineXSection",
			"XSectionType:="	, "None",
			"XSectionOrient:="	, "Auto",
			"XSectionWidth:="	, "0mm",
			"XSectionTopWidth:="	, "0mm",
			"XSectionHeight:="	, "0mm",
			"XSectionNumSegments:="	, "0",
			"XSectionBendType:="	, "Corner"
		]
	], 
	[
		"NAME:Attributes",
		"Name:="		, "StatorToothIntersector2",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.SweepAlongVector(
	[
		"NAME:Selections",
		"Selections:="		, StatorToothIntersector2,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:VectorSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepVectorX:="	, "0mm",
		"SweepVectorY:="	, "StatorAxialThickness",
		"SweepVectorZ:="	, "0mm"
	])

oEditor.SweepAlongVector(
	[
		"NAME:Selections",
		"Selections:="		, StatorToothIntersector2,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:VectorSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepVectorX:="	, "-4mm",
		"SweepVectorY:="	, "0mm",
		"SweepVectorZ:="	, "0mm"
	])

oEditor.Move(
	[
		"NAME:Selections",
		"Selections:="		, StatorToothIntersector2,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:TranslateParameters",
		"TranslateVectorX:="	, "-StatorPoleTeethStartX",
		"TranslateVectorY:="	, "0mm",
		"TranslateVectorZ:="	, "-StatorPoleWidth/2"
	])

oEditor.Rotate(
	[
		"NAME:Selections",
		"Selections:="		, StatorToothIntersector2,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:RotateParameters",
		"RotateAxis:="		, "Y",
		"RotateAngle:="		, "-StatorPoleTeethAngle"
	])

oEditor.Move(
	[
		"NAME:Selections",
		"Selections:="		, StatorToothIntersector2,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:TranslateParameters",
		"TranslateVectorX:="	, "StatorPoleTeethStartX",
		"TranslateVectorY:="	, "0mm",
		"TranslateVectorZ:="	, "StatorPoleWidth/2"
	])

StatorToothList=[StatorTooth,StatorToothIntersector1,StatorToothIntersector2]

oEditor.Intersect(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, StatorToothList))
	], 
	[
		"NAME:IntersectParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

print("Create " + str(StatorTooth) + " successful")

StatorToothDul=oEditor.DuplicateMirror(
	[
		"NAME:Selections",
		"Selections:="		, StatorTooth,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateToMirrorParameters",
		"DuplicateMirrorBaseX:=", "0mm",
		"DuplicateMirrorBaseY:=", "0mm",
		"DuplicateMirrorBaseZ:=", "0mm",
		"DuplicateMirrorNormalX:=", "0mm",
		"DuplicateMirrorNormalY:=", "0mm",
		"DuplicateMirrorNormalZ:=", "-1mm"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

StatorToothUnionList = [StatorPole,StatorTooth]
StatorToothUnionList.extend(StatorToothDul)

print(StatorToothUnionList)

oEditor.Unite(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, StatorToothUnionList))
	], 
	[
		"NAME:UniteParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

StatorToothUnion = oEditor.DuplicateAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, StatorToothUnionList[0],
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateAroundAxisParameters",
		"CreateNewObjects:="	, True,
		"WhichAxis:="		, "Y",
		"AngleStr:="		, "30deg",
		"NumClones:="		, "12"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

StatorList=[Stator,StatorToothUnionList[0]]
StatorList.extend(StatorToothUnion)

oEditor.Unite(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, StatorList))
	], 
	[
		"NAME:UniteParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

print("Create " + str(StatorList) + " successful")



###################################################################
#Create Armature Windings

ArmatureWindingsubtractor = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "-StatorPoleWidth/2-0.3mm",
		"YStart:="		, "-StatorAxialThickness/2-0.3mm",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorPoleWidth+0.6mm",
		"Height:="		, "StatorAxialThickness+0.6mm",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "ArmatureWindingsubtractor",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"copper\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

ArmatureWinding = oEditor.CreateRectangle(
	[
		"NAME:RectangleParameters",
		"IsCovered:="		, True,
		"XStart:="		, "-StatorPoleWidth/2-0.3mm-WindingThickness",
		"YStart:="		, "-StatorAxialThickness/2-0.3mm-WindingThickness",
		"ZStart:="		, "0mm",
		"Width:="		, "StatorPoleWidth+0.6mm+2*WindingThickness",
		"Height:="		, "StatorAxialThickness+0.6mm+2*WindingThickness",
		"WhichAxis:="		, "Z"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "ArmatureWinding",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 0,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"copper\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

oEditor.Subtract(
	[
		"NAME:Selections",
		"Blank Parts:="		, ArmatureWinding,
		"Tool Parts:="		, ArmatureWindingsubtractor
	], 
	[
		"NAME:SubtractParameters",
		"KeepOriginals:="	, False,
		"TurnOnNBodyBoolean:="	, True
	])

oEditor.SweepAlongVector(
	[
		"NAME:Selections",
		"Selections:="		, ArmatureWinding,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:VectorSweepParameters",
		"DraftAngle:="		, "0deg",
		"DraftType:="		, "Round",
		"CheckFaceFaceIntersection:=", False,
		"ClearAllIDs:="		, False,
		"SweepVectorX:="	, "0mm",
		"SweepVectorY:="	, "0",
		"SweepVectorZ:="	, "WindingRadialLength"
	])

oEditor.Move(
	[
		"NAME:Selections",
		"Selections:="		, ArmatureWinding,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:TranslateParameters",
		"TranslateVectorX:="	, "0mm",
		"TranslateVectorY:="	, "0mm",
		"TranslateVectorZ:="	, "StatorPMOuterRadius+0.5mm"
	])

ArmatureWindingDup = oEditor.DuplicateAroundAxis(
	[
		"NAME:Selections",
		"Selections:="		, ArmatureWinding,
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:DuplicateAroundAxisParameters",
		"CreateNewObjects:="	, True,
		"WhichAxis:="		, "Y",
		"AngleStr:="		, "30deg",
		"NumClones:="		, "12"
	], 
	[
		"NAME:Options",
		"DuplicateAssignments:=", True
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

ArmatureWindingList=[ArmatureWinding]
ArmatureWindingList.extend(ArmatureWindingDup)

print("Create " + str(ArmatureWindingList) + " successful")

ArmatureWindingSectionList= oEditor.Section(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, ArmatureWindingList)),
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"NAME:SectionToParameters",
		"CreateNewObjects:="	, True,
		"SectionPlane:="	, "ZX",
		"SectionCrossObject:="	, False
	])

# print(ArmatureWindingSectionList)

ArmatureWindingSectionSeparateList = oEditor.SeparateBody(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, ArmatureWindingSectionList)),
		"NewPartsModelFlag:="	, "Model"
	], 
	[
		"CreateGroupsForNewObjects:=", False
	])

# Create a new list with strings ending with "Separate1"
ArmatureWindingSectionDeleteList = [s for s in ArmatureWindingSectionSeparateList if s.endswith("Separate1")]

# print(ArmatureWindingSectionDeleteList)

oEditor.Delete(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, ArmatureWindingSectionDeleteList))
	])

##################################################################
#Create Suspension Windings



################################################################
# air boundry, band, air gap segment

Air = oEditor.CreateCylinder(
	[
		"NAME:CylinderParameters",
		"XCenter:="		, "0mm",
		"YCenter:="		, "-20mm",
		"ZCenter:="		, "0mm",
		"Radius:="		, "70mm",
		"Height:="		, "40mm",
		"WhichAxis:="		, "Y",
		"NumSides:="		, "0"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "Air",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 1,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

Band = oEditor.CreateCylinder(
	[
		"NAME:CylinderParameters",
		"XCenter:="		, "0mm",
		"YCenter:="		, "-13mm",
		"ZCenter:="		, "0mm",
		"Radius:="		, "StatorInnerRadius/2+RotorOuterRadius/2",
		"Height:="		, "26mm",
		"WhichAxis:="		, "Y",
		"NumSides:="		, "0"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "Band",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 1,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

Airgap1 = oEditor.CreateCylinder(
	[
		"NAME:CylinderParameters",
		"XCenter:="		, "0mm",
		"YCenter:="		, "-12mm",
		"ZCenter:="		, "0mm",
		"Radius:="		, "1*StatorInnerRadius/8+7*RotorOuterRadius/8",
		"Height:="		, "24mm",
		"WhichAxis:="		, "Y",
		"NumSides:="		, "0"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "Airgap1",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 1,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

Airgap2 = oEditor.CreateCylinder(
	[
		"NAME:CylinderParameters",
		"XCenter:="		, "0mm",
		"YCenter:="		, "-12mm",
		"ZCenter:="		, "0mm",
		"Radius:="		, "3*StatorInnerRadius/8+5*RotorOuterRadius/8",
		"Height:="		, "24mm",
		"WhichAxis:="		, "Y",
		"NumSides:="		, "0"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "Airgap2",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 1,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

Airgap3 = oEditor.CreateCylinder(
	[
		"NAME:CylinderParameters",
		"XCenter:="		, "0mm",
		"YCenter:="		, "-12mm",
		"ZCenter:="		, "0mm",
		"Radius:="		, "4*StatorInnerRadius/8+4*RotorOuterRadius/8",
		"Height:="		, "24mm",
		"WhichAxis:="		, "Y",
		"NumSides:="		, "0"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "Airgap3",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 1,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

Airgap4 = oEditor.CreateCylinder(
	[
		"NAME:CylinderParameters",
		"XCenter:="		, "0mm",
		"YCenter:="		, "-12mm",
		"ZCenter:="		, "0mm",
		"Radius:="		, "6*StatorInnerRadius/8+2*RotorOuterRadius/8",
		"Height:="		, "24mm",
		"WhichAxis:="		, "Y",
		"NumSides:="		, "0"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "Airgap4",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 1,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

Airgap5 = oEditor.CreateCylinder(
	[
		"NAME:CylinderParameters",
		"XCenter:="		, "0mm",
		"YCenter:="		, "-12mm",
		"ZCenter:="		, "0mm",
		"Radius:="		, "8*StatorInnerRadius/8+0*RotorOuterRadius/8",
		"Height:="		, "24mm",
		"WhichAxis:="		, "Y",
		"NumSides:="		, "0"
	], 
	[
		"NAME:Attributes",
		"Name:="		, "Airgap5",
		"Flags:="		, "",
		"Color:="		, "(143 175 143)",
		"Transparency:="	, 1,
		"PartCoordinateSystem:=", "Global",
		"UDMId:="		, "",
		"MaterialValue:="	, "\"vacuum\"",
		"SurfaceMaterialValue:=", "\"\"",
		"SolveInside:="		, True,
		"ShellElement:="	, False,
		"ShellElementThickness:=", "0mm",
		"ReferenceTemperature:=", "20cel",
		"IsMaterialEditable:="	, True,
		"UseMaterialAppearance:=", False,
		"IsLightweight:="	, False
	])

AirgapSubtractorList=[Airgap1,Airgap2,Airgap3,Airgap4,Airgap5]

oEditor.Subtract(
	[
		"NAME:Selections",
		"Blank Parts:="		, AirgapSubtractorList[4],
		"Tool Parts:="		, AirgapSubtractorList[3]
	], 
	[
		"NAME:SubtractParameters",
		"KeepOriginals:="	, True,
		"TurnOnNBodyBoolean:="	, True
	])

oEditor.Subtract(
	[
		"NAME:Selections",
		"Blank Parts:="		, AirgapSubtractorList[3],
		"Tool Parts:="		, AirgapSubtractorList[2]
	], 
	[
		"NAME:SubtractParameters",
		"KeepOriginals:="	, True,
		"TurnOnNBodyBoolean:="	, True
	])

oEditor.Subtract(
	[
		"NAME:Selections",
		"Blank Parts:="		, AirgapSubtractorList[1],
		"Tool Parts:="		, AirgapSubtractorList[0]
	], 
	[
		"NAME:SubtractParameters",
		"KeepOriginals:="	, True,
		"TurnOnNBodyBoolean:="	, True
	])

AirgapList=[AirgapSubtractorList[4],AirgapSubtractorList[3],AirgapSubtractorList[1]]

AirgapDeleteList=[AirgapSubtractorList[2],AirgapSubtractorList[0]]

oEditor.Delete(
	[
		"NAME:Selections",
		"Selections:="		, ",".join(map(str, AirgapDeleteList))
	])

print("Create " + str(AirgapList) + " successful")



desktop.save_project()
##################################################### Mesh Operation

oModule = oDesign.GetModule("MeshSetup")

oModule.InitialMeshSettings(
	[
		"NAME:MeshSettings",
		[
			"NAME:GlobalSurfApproximation",
			"CurvedSurfaceApproxChoice:=", "UseSlider",
			"SliderMeshSettings:="	, 8
		],
		[
			"NAME:GlobalCurvilinear",
			"Apply:="		, True
		],
		[
			"NAME:GlobalModelRes",
			"UseAutoLength:="	, True
		],
		"MeshMethod:="		, "Auto",
		"UseLegacyFaceterForTauVolumeMesh:=", False,
		"DynamicSurfaceResolution:=", False,
		"UseFlexMeshingForTAUvolumeMesh:=", False,
		"UseAlternativeMeshMethodsAsFallBack:=", True,
		"AllowPhiForLayeredGeometry:=", False
	])


TotalPMList = RotorRadialPMList+RotorAxialPMList+StatorAxialPMList
TotalIronList = RotorAxialIronList+StatorAxialIronList

SteelList=[Rotor]
SteelList.append(Stator)


# HBCPM.mesh.assign_length_mesh(assignment=AirgapList, maximum_length=0.7, maximum_elements=None, name="Airgap")

# HBCPM.mesh.assign_length_mesh(assignment=ArmatureWindingList, maximum_length=2, maximum_elements=None, name="Winding")

# HBCPM.mesh.assign_length_mesh(assignment=TotalPMList, maximum_length=1, maximum_elements=None, name="PMs")

# HBCPM.mesh.assign_length_mesh(assignment=TotalPMList, maximum_length=1, maximum_elements=None, name="PMs")

# HBCPM.mesh.assign_length_mesh(assignment=TotalIronList, maximum_length=1, maximum_elements=None, name="Iron")

# HBCPM.mesh.assign_length_mesh(assignment=Rotor, maximum_length=2.5, maximum_elements=None, name="Rotor")

# HBCPM.mesh.assign_length_mesh(assignment=Stator, maximum_length=2.5, maximum_elements=None, name="Stator")

oModule.AssignLengthOp(
	[
		"NAME:Airgap",
		"RefineInside:="	, True,
		"Enabled:="		, True,
		"Objects:="		, AirgapList,
		"RestrictElem:="	, False,
		"NumMaxElem:="		, "1000",
		"RestrictLength:="	, True,
		"MaxLength:="		, "0.7mm"
	])

oModule.AssignLengthOp(
	[
		"NAME:Winding",
		"RefineInside:="	, True,
		"Enabled:="		, True,
		"Objects:="		, ArmatureWindingList,
		"RestrictElem:="	, False,
		"NumMaxElem:="		, "1000",
		"RestrictLength:="	, True,
		"MaxLength:="		, "2mm"
	])

oModule.AssignLengthOp(
	[
		"NAME:PM",
		"RefineInside:="	, True,
		"Enabled:="		, True,
		"Objects:="		, TotalPMList,
		"RestrictElem:="	, False,
		"NumMaxElem:="		, "1000",
		"RestrictLength:="	, True,
		"MaxLength:="		, "1mm"
	])

oModule.AssignLengthOp(
	[
		"NAME:Iron",
		"RefineInside:="	, True,
		"Enabled:="		, True,
		"Objects:="		, TotalIronList,
		"RestrictElem:="	, False,
		"NumMaxElem:="		, "1000",
		"RestrictLength:="	, True,
		"MaxLength:="		, "1mm"
	])

oModule.AssignLengthOp(
	[
		"NAME:Steel",
		"RefineInside:="	, True,
		"Enabled:="		, True,
		"Objects:="		, SteelList,
		"RestrictElem:="	, False,
		"NumMaxElem:="		, "1000",
		"RestrictLength:="	, True,
		"MaxLength:="		, "3mm"
	])

###################################################################
# Set force and torque

oModule = oDesign.GetModule("MaxwellParameterSetup")

RotorList=RotorAxialIronList+RotorRadialPMList+RotorAxialPMList
RotorList.append(Rotor)

oModule.AssignForce(
	[
		"NAME:Force",
		"Reference CS:="	, "Global",
		"Is Virtual:="		, True,
		"Objects:="		, RotorList
	])
oModule.AssignTorque(
	[
		"NAME:TorqueRotation",
		"Is Virtual:="		, True,
		"Coordinate System:="	, "Global",
		"Axis:="		, "Y",
		"Is Positive:="		, True,
		"Objects:="		, RotorList
	])
oModule.AssignTorque(
	[
		"NAME:TorqueTilt",
		"Is Virtual:="		, True,
		"Coordinate System:="	, "Global",
		"Axis:="		, "X",
		"Is Positive:="		, True,
		"Objects:="		, RotorList
	])

AirSurfaceList = HBCPM.modeler.get_object_faces(assignment=Air)

desktop.save_project()


PyAEDT INFO: Python version 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:07:43) [MSC v.1942 64 bit (AMD64)]
PyAEDT INFO: PyAEDT version 0.12.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\shins\AppData\Local\Temp\pyaedt_shins_6c804f6f-4a47-4bfa-b720-5fa48f7178f1.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 50114
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v241\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.1 version started with process ID 38092.
Instance of an Aedt object:13
PyAEDT INFO: Python version 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:07:43) [MSC v.1942 64 bit (AMD64)]
PyAEDT INFO: PyAEDT version 0.12.0.
PyAEDT INFO: Returning found Desktop session with PID 38092!
PyAE

GrpcApiError: Failed to execute gRPC AEDT command: Rename

In [13]:

desktop.release_desktop()

PyAEDT ERROR: **************************************************************
PyAEDT ERROR:   File "c:\Users\shins\anaconda3\envs\PyMotor\lib\runpy.py", line 196, in _run_module_as_main
PyAEDT ERROR:     return _run_code(code, main_globals, None,
PyAEDT ERROR:   File "c:\Users\shins\anaconda3\envs\PyMotor\lib\runpy.py", line 86, in _run_code
PyAEDT ERROR:     exec(code, run_globals)
PyAEDT ERROR:   File "c:\Users\shins\anaconda3\envs\PyMotor\lib\asyncio\base_events.py", line 603, in run_forever
PyAEDT ERROR:     self._run_once()
PyAEDT ERROR:   File "c:\Users\shins\anaconda3\envs\PyMotor\lib\asyncio\base_events.py", line 1909, in _run_once
PyAEDT ERROR:     handle._run()
PyAEDT ERROR:   File "c:\Users\shins\anaconda3\envs\PyMotor\lib\asyncio\events.py", line 80, in _run
PyAEDT ERROR:     self._context.run(self._callback, *self._args)
PyAEDT ERROR:   File "C:\Users\shins\AppData\Local\Temp\ipykernel_35384\171691216.py", line 1, in <module>
PyAEDT ERROR:     desktop.release_desktop()
PyAE

False